In [13]:
import numpy as np

__author__ = "Jared Thompson"


class GradientDescent(object):
    def __init__(self, fit_intercept=True, normalize=False, gradient=None, mu=None, sigma=None, ):
        '''
        INPUT: GradientDescent, boolean
        OUTPUT: None
        Initialize class variables. cost is the function used to compute the
        cost.
        '''
        self.coeffs = None
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.mu = mu
        self.sigma = sigma
        self.alpha = None
        self.gradient = gradient

    def run(self, X, y, coeffs=None, alpha=0.01, num_iterations=100):
        self.calculate_normalization_factors(X)
        X = self.maybe_modify_matrix(X)
        (self.coeffs,self.alpha) = (coeffs, alpha)
        (M, N) = ( float(d) for d in X.shape)
        if not np.any(self.coeffs):
            self.coeffs = np.zeros(N)
      
        if self.fit_intercept:
            self.coeffs = np.insert(self.coeffs, 0, 0)
        
        for i in xrange(num_iterations):
            self.coeffs += alpha * self.gradient(X, y, self.coeffs)

    def calculate_normalization_factors(self, X):
        '''
        INPUT: GradientDescent, 2 dimensional numpy array
        OUTPUT: None
        Initialize mu and sigma instance variables to be the numpy arrays
        containing the mean and standard deviation for each column of X.
        '''
        self.mu = np.average(X, 0)
        self.sigma = np.std(X, 0)
        # Don't normalize intercept column
        self.mu[self.sigma == 0] = 0
        self.sigma[self.sigma == 0] = 1

    def add_intercept(self, X):
        '''
        INPUT: 2 dimensional numpy array
        OUTPUT: 2 dimensional numpy array
        Return a new 2d array with a column of ones added as the first
        column of X.
        '''
        return np.hstack((np.ones((X.shape[0], 1)), X))        

    def maybe_modify_matrix(self, X):
        '''
        INPUT: GradientDescent, 2 dimensional numpy array
        OUTPUT: 2 dimensional numpy array
        Depending on the settings, normalizes X and adds a feature for the
        intercept.
        '''
        if self.normalize:
            X = (X - self.mu) / self.sigma
        if self.fit_intercept:
            return self.add_intercept(X)
        return X




In [45]:
import numpy as np

__author__ = "Jared Thompson"

class LogisticRegression(object):

    def __init__(self, fit_intercept = True, scale = True, norm = "L2"):
        '''
        INPUT: GradientDescent, function, function, function
        OUTPUT: None

        Initialize class variables. Takes three functions:
        cost: the cost function to be minimized
        gradient: function to calculate the gradient of the cost function
        predict: function to calculate the predicted values (0 or 1) for
        the given data
        '''
        gradient_choices = {None: self.cost_gradient, "L1": self.cost_gradient_lasso, "L2": self.cost_gradient_ridge}

        self.alpha = None
        self.gamma = None
        self.coeffs = None
        self.num_iterations = 0
        self.fit_intercept = fit_intercept
        self.scale = scale
        self.normalize = False
        if norm:
            self.norm = norm
            self.normalize = True
        self.gradient = gradient_choices[norm]        
        
    def fit(self,  X, y, alpha=0.01, num_iterations=10000, gamma=0.):
        '''
        INPUT: 2 dimensional numpy array, numpy array, float, int, float
        OUTPUT: numpy array

        Main routine to train the model coefficients to the data
        the given coefficients.
        '''
        (M, N) = (float(x) for x in X.shape)
        (self.alpha, self.gamma) = (alpha, gamma)
        self.num_iterations += num_iterations
        self.coeffs = np.random.randn(N)
        gradient = GradientDescent(self.fit_intercept, self.normalize, self.gradient)
        gradient.run(X, y, self.coeffs, self.alpha, num_iterations)
        self.coeffs = gradient.coeffs
        
    def predict(self, X):
        '''
        INPUT: 2 dimensional numpy array, numpy array
        OUTPUT: numpy array

        Calculate the predicted values (0 or 1) for the given data with
        the given coefficients.
        '''
        return np.around(self.hypothesis(X, self.coeffs)).astype(bool)

    def hypothesis(self, X, coeffs):
        '''
        INPUT: 2 dimensional numpy array, numpy array
        OUTPUT: numpy array

        Calculate the predicted percentages (floats between 0 and 1)
        for the given data with the given coefficients.
        '''  
        return 1. / (1. + np.exp(-X.dot(coeffs)))

    def cost_function(self, X, y, coeffs):
        '''
        INPUT: 2 dimensional numpy array, numpy array, numpy array
        OUTPUT: float

        Calculate the value of the cost function for the data with the
        given coefficients.
        '''
        h = self.hypothesis(X, coeffs)
        (M, N) = (float(x) for x in X.shape)
        return (1./M)* (y.dot(h) + (1 - y).dot(1 - h))

    def cost_lasso(self, X, y, coeffs):
        '''
        INPUT: 2 dimensional numpy array, numpy array, numpy array
        OUTPUT: float

        Calculate the value of the cost function with lasso regularization
        for the data with the given coefficients.
        '''
        h = self.hypothesis(X, coeffs)
        (M, N) = (float(x) for x in X.shape)
        return (1./M) * (y.dot(h) + (1 - y).dot(1 - h) + self.gamma * np.sum([np.abs(c) for c in coeffs]))

    def cost_ridge(self, X, y, coeffs):
        '''
        INPUT: 2 dimensional numpy array, numpy array, numpy array
        OUTPUT: float

        Calculate the value of the cost function with ridge regularization
        for the data with the given coefficients.
        '''
        h = self.hypothesis(X, coeffs)
        (M, N) = (float(x) for x in X.shape)
        return (1./M) * (y.dot(h) + (1 - y).dot(1 - h) + self.gamma * coeffs.dot(coeffs.T) / 2)

    def cost_gradient(self, X, y, coeffs):
        '''
        INPUT: 2 dimensional numpy array, numpy array, numpy array
        OUTPUT: numpy array

        Calculate the gradient of the cost function at the given value
        for the coeffs.

        Return an array of the same size as the coeffs array.
        '''
        h = self.hypothesis(X, coeffs)
        return X.T.dot(y - h)

    def cost_gradient_lasso(self, X, y, coeffs):
        '''
        INPUT: 2 dimensional numpy array, numpy array, numpy array
        OUTPUT: numpy array

        Calculate the gradient of the cost function with regularization
        at the given value for the coeffs.

        Return an array of the same size as the coeffs array.
        '''
        h = self.hypothesis(X, coeffs)
        weights = [c/np.abs(c) for c in coeffs[1:]]
        weights.insert(0, 0)
        return X.T.dot(y - h) + self.gamma * weights / N

    def cost_gradient_ridge(self, X, y, coeffs):
        '''
        INPUT: 2 dimensional numpy array, numpy array, numpy array
        OUTPUT: numpy array

        Calculate the gradient of the cost function with regularization
        at the given value for the coeffs.

        Return an array of the same size as the coeffs array.
        '''
        (M, N) = (float(x) for x in X.shape)
        h = self.hypothesis(X, coeffs)
        weights = coeffs[1:]
        weights = np.insert(weights, 0, 0)
        return X.T.dot(y - h) + self.gamma * weights / N

In [46]:
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import numpy as np
X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                       n_clusters_per_class=2, n_samples=1000)

X_train, X_test, y_train, y_test = train_test_split(X, y)
model = LogisticRegression()
model.fit(X_train, y_train, gamma=0.1)

X_test = np.hstack((np.zeros((X_test.shape[0], 1)), X_test)) 
probabilities = model.predict(X_test)
fpr, tpr, thresholds = roc_curve(probabilities, y_test)

plt.plot(np.asarray(fpr), np.asarray(tpr))
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel("False Positive Rate (1 - Specificity)")
plt.ylabel("True Positive Rate (Sensitivity, Recall)")
plt.title("ROC plot of fake data")
plt.show()

<bound method LogisticRegression.cost_gradient_ridge of <__main__.LogisticRegression object at 0x108410190>>


In [41]:
def roc_curve(probabilities, labels):
    '''
    INPUT: numpy array, numpy array
    OUTPUT: list, list, list

    Take a numpy array of the predicted probabilities and a numpy array of theau
    true labels.
    Return the True Positive Rates, False Positive Rates and Thresholds for the
    ROC curve.
    '''

    '''#Following Algorithm 2 from
    http://www.hpl.hp.com/techreports/2003/HPL-2003-4.pdf '''

    '''#Initialize lists'''
    TPRs = []
    FPRs = []
    treshholds = []

    '''#Count number of positive and negative cases'''
    num_pos_cases = list(labels).count(1)
    num_neg_cases = list(labels).count(0)

    '''#Sort probabilities by desc order'''
    sorted_probabilities = sorted(probabilities, reverse=True)

    '''#Sort indexes of probabilities by desc order,
    this will help to match indices of labels'''
    sorted_indices = np.argsort(probabilities)[::-1]
    sorted_labels = labels[sorted_indices]

    '''#Intialize true positives and false positives counter'''
    '''#true positives'''
    tp = 0.0
    '''#false positives'''
    fp = 0.0
    '''#auxiliary var'''
    prev_prob = float("inf")

    for prob in sorted_probabilities:

        '''#Append values of prob into treshholds list '''
        treshold = prob
        treshholds.append(treshold)

        for i, label in enumerate(sorted_labels):

            '''#Check if prob of label is not equal to treshold '''
            if probabilities[i] != prev_prob:
                '''#if the index is not equal to zero then
                append values into TPRs and FPRs lists'''
                if i != 0:
                    TPRs.append(tp/num_pos_cases)
                    FPRs.append(fp/num_neg_cases)
                '''Replace value of prev_prob with  prob of label[i] '''
                prev_prob = probabilities[i]

            '''#If label is positive then increase the counter of tp ,
            otherwise increase the fp counter'''
            if label == 1:
                tp += 1.0
            else:
                fp += 1.0

        '''#Calculate true positives rate and false negative rate '''

        '''#true_positives= number correctly predicted
        true positives/ number of positive cases'''
        TPRs.append(tp/num_pos_cases)

        '''#true_positives= number incorrectly predicted true positives/
        number of negative cases'''
        FPRs.append(fp/num_neg_cases)

    return FPRs, TPRs, treshholds

In [47]:
(a, b, c) = (1, 2, 3)

In [48]:
a


1